# Triplet Loss

### Load Train, Test, and Validation data

In [1]:
import numpy as np

In [2]:
train_features_path =  "../feature/BENGALI-feature/train_features.npy"
train_names_path = "../feature/BENGALI-feature/train_names.npy"
train_labels_path = "../feature/BENGALI-feature/train_labels.npy"

test_features_path = "../feature/BENGALI-feature/test_features.npy"
test_names_path = "../feature/BENGALI-feature/test_names.npy"
test_labels_path = "../feature/BENGALI-feature/test_labels.npy"

validation_features_path = "../feature/BENGALI-feature/validation_features.npy"
validation_names_path = "../feature/BENGALI-feature/validation_names.npy"
validation_labels_path = "../feature/BENGALI-feature/validation_labels.npy"

gallery_feature_path = "../feature/BENGALI-feature/gallery_features.npy"
gallery_name_path = "../feature/BENGALI-feature/gallery_names.npy"

train_features = np.load(train_features_path)
train_names = np.load(train_names_path)
train_labels = np.load(train_labels_path)

test_features = np.load(test_features_path)
test_names = np.load(test_names_path)
test_labels = np.load(test_labels_path)

validation_features = np.load(validation_features_path)
validation_names = np.load(validation_names_path)
validation_labels = np.load(validation_labels_path)

gallery_features = np.load(gallery_feature_path)
gallery_names = np.load(gallery_name_path)

In [3]:
print(train_features.shape)
print(train_labels.shape)
print(train_names.shape)
print(test_features.shape)
print(test_labels.shape)
print(test_names.shape)
print(validation_features.shape)
print(validation_labels.shape)
print(validation_names.shape)
print(gallery_features.shape)
print(gallery_names.shape)

(3200, 512)
(3200,)
(3200,)
(1100, 512)
(1100,)
(1100,)
(1100, 512)
(1100,)
(1100,)
(100, 512)
(100,)


### Creating Triplets

In [4]:
unique_names = set(train_names.flatten())
sorted_names = sorted(unique_names)

In [5]:
print(sorted_names)

['001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '047', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', '060', '061', '062', '063', '064', '065', '066', '067', '068', '069', '070', '071', '072', '073', '074', '075', '076', '077', '078', '079', '080', '081', '082', '083', '084', '085', '086', '087', '088', '089', '090', '091', '092', '093', '094', '095', '096', '097', '098', '099', '100']


In [6]:
print(gallery_names)

['001' '002' '003' '004' '005' '006' '007' '008' '009' '010' '011' '012'
 '013' '014' '015' '016' '017' '018' '019' '020' '021' '022' '023' '024'
 '025' '026' '027' '028' '029' '030' '031' '032' '033' '034' '035' '036'
 '037' '038' '039' '040' '041' '042' '043' '044' '045' '046' '047' '048'
 '049' '050' '051' '052' '053' '054' '055' '056' '057' '058' '059' '060'
 '061' '062' '063' '064' '065' '066' '067' '068' '069' '070' '071' '072'
 '073' '074' '075' '076' '077' '078' '079' '080' '081' '082' '083' '084'
 '085' '086' '087' '088' '089' '090' '091' '092' '093' '094' '095' '096'
 '097' '098' '099' '100']


In [7]:
def create_triplets(unique_names, gallery_features, features, labels):
    anchors, positives, negatives = [], [], []
    for name, gallery in zip(unique_names, gallery_features):
        # find all the instances for name
        indices = np.where(train_names == name)[0]
        # print(indices)
        
        # separate instances to positive and negative
        positive_set, negative_set = [], []
        for index in indices:
            # print(index)
            if labels[index] == 0:
                negative_set.append(features[index])
                # print(features[index])
            else:
                positive_set.append(features[index]) 

        negative_set = np.array(negative_set)
        positive_set = np.array(positive_set)

        # use feature gallery for anchor
        # print(gallery.size)
        anchor = gallery
        
        # print(positive_set.size)
        # print(negative_set.size)

        # randomize data picks to accomodate for unequal data size (total size is based on negative data size due to it being bigger than positive data size)
        positive_idx = np.random.choice(len(positive_set), size=len(negative_set), replace=True)
        positive = positive_set[positive_idx]
        
        anchor_set = []
        for _ in range(0, len(negative_set)):
            anchor_set.append(anchor)

        anchors.append(anchor_set)
        positives.append(positive)
        negatives.append(negative_set)
        # print(len(anchors))
        # print(len(positives))
        # print(len(negatives))

    anchors = np.concatenate(anchors, axis=0)
    
    positives = np.concatenate(positives, axis=0)

    negatives = np.concatenate(negatives, axis=0)
    
    return anchors, positives, negatives

In [8]:
anchors, positives, negatives = create_triplets(sorted_names, gallery_features, train_features, train_labels)

In [9]:
print("Anchors shape:", anchors.shape)
print("Positives shape:", positives.shape)
print("Negatives shape:", negatives.shape)

Anchors shape: (1800, 512)
Positives shape: (1800, 512)
Negatives shape: (1800, 512)


### Triplet Loss Function

In [10]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [11]:
input_shape = (train_features.shape[1],)

In [12]:
embedding_model = models.Sequential([
    layers.Input(shape=input_shape),
    layers.Dense(512, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [13]:
def create_triplet_loss_model(embedding_model, input_shape):
    anchor_input = layers.Input(shape=input_shape, name='anchor_input')
    positive_input = layers.Input(shape=input_shape, name='positive_input')
    negative_input = layers.Input(shape=input_shape, name='negative_input')

    # Get embeddings
    anchor_embedding = embedding_model(anchor_input)
    positive_embedding = embedding_model(positive_input)
    negative_embedding = embedding_model(negative_input)

    # Triplet loss function
    margin = 0.2
    positive_distance = tf.reduce_sum(tf.square(anchor_embedding - positive_embedding), axis=1)
    negative_distance = tf.reduce_sum(tf.square(anchor_embedding - negative_embedding), axis=1)
    loss = tf.maximum(0.0, positive_distance - negative_distance + margin)
    loss = tf.reduce_mean(loss)

    triplet_loss_model = models.Model(inputs=[anchor_input, positive_input, negative_input], outputs=loss)
    return triplet_loss_model

In [14]:
embedding_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               262656    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 427009 (1.63 MB)
Trainable params: 427009 (1.63 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
triplet_model = create_triplet_loss_model(embedding_model, input_shape)
triplet_model.compile(optimizer='adam', loss='mean_squared_error')

In [16]:
triplet_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 anchor_input (InputLayer)   [(None, 512)]                0         []                            
                                                                                                  
 positive_input (InputLayer  [(None, 512)]                0         []                            
 )                                                                                                
                                                                                                  
 negative_input (InputLayer  [(None, 512)]                0         []                            
 )                                                                                                
                                                                                              

In [17]:
triplet_model.fit([anchors, positives, negatives], np.zeros_like(anchors), epochs=10, batch_size=32)

Epoch 1/10

57/57 [==============================] - 3s 8ms/step - loss: 0.0126
Epoch 2/10
57/57 [==============================] - 0s 8ms/step - loss: 0.0047
Epoch 3/10
57/57 [==============================] - 0s 8ms/step - loss: 0.0021
Epoch 4/10
57/57 [==============================] - 0s 8ms/step - loss: 9.9096e-04
Epoch 5/10
57/57 [==============================] - 0s 8ms/step - loss: 0.0027
Epoch 6/10
57/57 [==============================] - 0s 8ms/step - loss: 0.0010
Epoch 7/10
57/57 [==============================] - 0s 8ms/step - loss: 6.3359e-04
Epoch 8/10
57/57 [==============================] - 0s 8ms/step - loss: 5.3216e-04
Epoch 9/10
57/57 [==============================] - 0s 8ms/step - loss: 3.1062e-04
Epoch 10/10
57/57 [==============================] - 0s 8ms/step - loss: 2.4155e-04


In [18]:
from sklearn.metrics import accuracy_score

In [19]:
print(test_labels)

[1 1 1 ... 0 0 0]


In [20]:
predictions = embedding_model.predict(test_features)

35/35 [==============================] - 0s 5ms/step


In [23]:
binary_predictions = (predictions >= 0.5).astype(int)
accuracy = accuracy_score(test_labels, binary_predictions)
print(f'Test Accuracy: {accuracy}')

Test Accuracy: 0.17454545454545456
